In [1]:
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon, Point, LineString
import pandas as pd
from itertools import combinations

%matplotlib inline